# Data Ingestion

In [1]:
import os

In [2]:
%pwd

'/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/bbc-news-sorting/notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/bbc-news-sorting'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngetsionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.news_sorting_project.constants import *
from src.news_sorting_project.utils.common import read_yaml, create_directories

In [7]:
class CofigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngetsionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngetsionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import boto3
import urllib.request as request
import zipfile
from src.news_sorting_project.utils.common import get_size
from src.news_sorting_project import logger
from dotenv import load_dotenv
import os
load_dotenv()

True

In [9]:
# Retrieve the credentials from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_default_region = os.getenv("AWS_DEFAULT_REGION")

# Define the S3 bucket and object
bucket_name = os.getenv("AWS_BUCKET_NAME")
object_key = os.getenv("AWS_OBJECT_KEY")

In [10]:
class DataIngetsion:
    def __init__(self, config: DataIngetsionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Initialize a session using Boto3 with the credentials from the environment
            s3_client = boto3.client(
                's3',
                aws_access_key_id=aws_access_key_id,
                aws_secret_access_key=aws_secret_access_key,
                region_name=aws_default_region
            )

            s3_client.download_file(bucket_name, object_key, self.config.local_data_file)

            logger.info(f"{object_key} download! with following info: \n{get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extract the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = CofigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngetsion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

Above code initiates data ingestion, by downloading zip file from AWS S3 bucket and then extracts the same zip in `data ingestion` folder